In [8]:
from lxml import html
import lxml.html as lh
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
import time
from pymongo import MongoClient
import urllib

browser = webdriver.Firefox()
browser.get('http://www.agoda.com/zh-tw/minimi-inn-sail-rock-beach-house/hotel/kenting-tw.html?asq=8aQM%252fLMX6sQqpwNo%252bcT%252fesd5woCoEAsgdsoDlrTxVANFYiIJlMSJEqVM9k%252bhhYTlFk6HDFJ917cE2i2Iq%252bsqvRlOSQjjk7GSzZ8eMU0RwWQ776RNwdF2CGV9quyDD8KQn84Zs7nb3obspC103WF6oxlJI5IT3tUSRtqf%252fqhArWrY60MRtQ%252bQ3P5ekb2fajJr&cid=1732641&tag=41460a09-3e65-d173-1233-629e2428d88e&_ga=1.32921851.451805761.1461778654&checkin=2016-05-09&los=1&adults=2&childs=0&rooms=1&objectid=1032347&objecttypeid=2')
time.sleep(5)

# Get DB agoda ready
client = MongoClient()
# DB Name: agoda
db = client.s_1011213_agoda
# Clean the collection hotels
db.hotels.drop()
db.info.drop()
db.images.drop()
db.room_style.drop()
db.comment.drop()

def extract():
    content = browser.page_source # 將 網頁 的原始碼拿出來
    doc = lh.fromstring(content) # content現在還是一個物件 lh = lxml的html ,將它轉成string
    ta_divs = doc.cssselect('div.hotel-footer-panel')
    nawp_data = []
    nawp_dict = {}
    for ta in ta_divs:
        tb_divs = doc.cssselect('h1')[0].text_content()
        tc_divs = doc.cssselect('div.col-xs-9')[1].text_content()
        nawp_dict['area'] = ta.cssselect('span')[1].text_content() #hotel-info 下有很多個span 第一個是area 所以是[0]
        nawp_dict['room_num'] = ta.cssselect('span')[3].text_content()
        try: nawp_dict['wifi'] = ta.cssselect('span')[4].text_content() #使用try&except 因為不確定有沒有!!!! 
        except: nawp_dict['wifi'] = 'not available'
        nawp_data.append(nawp_dict)
        db.hotels.insert_one({'name':tb_divs ,'area': nawp_data[0]['area'],'room_num': nawp_data[0]['room_num'] ,'wifi':nawp_data[0]['wifi'],'info':tc_divs})

def extract_a():
    content = browser.page_source # 將 網頁 的原始碼拿出來
    doc = lh.fromstring(content) # content現在還是一個物件 lh = lxml的html ,將它轉成string
    tb_divs = doc.cssselect('h1')[0].text_content()
    tc_divs = doc.cssselect('div.info-container.collapsible-room')
    nc_data = []
    nc_dict = {}
    for tc in tc_divs:
        try: nc_dict['hot'] = tc.cssselect('i.icon-star')[0].text_content()
        except: nc_dict['hot'] = 'not hot'
        if nc_dict['hot'] == 'not hot':
            nc_dict['room_style'] = tc.cssselect('span')[0].text_content()
            nc_dict['en_room_style'] = tc.cssselect('span.eng-room-name')[0].text_content()
        else:
            nc_dict['hot'] = tc.cssselect('span')[0].text_content()
            nc_dict['room_style'] = tc.cssselect('span')[1].text_content()
            nc_dict['en_room_style'] = tc.cssselect('span.eng-room-name')[0].text_content()     
        nc_data.append(nc_dict)
        db.room_style.insert_one({'hot':nc_data[0]['hot'],'room_style':nc_data[0]['room_style'],'en_room_style':nc_data[0]['en_room_style']})

def extract_b():
    content = browser.page_source # 將 網頁 的原始碼拿出來
    doc = lh.fromstring(content)
    td_divs = doc.cssselect('li.list-item')
    np_data = []
    np_dict = {}
    for ta in td_divs:
            np_dict['service'] = ta.cssselect('li')[0].text_content() #hotel-info 下有很多個span 第一個是area 所以是[0]
            np_data.append(np_dict)
            db.info.insert_one({'service':np_data[0]['service']})

def extract_c():
    content = browser.page_source # 將 網頁 的原始碼拿出來
    doc = lh.fromstring(content) # content現在還是一個物件 lh = lxml的html ,將它轉成string
    ta_divs = doc.cssselect('div.col-xs-9.review-comment')
    nawp_data = []
    nawp_dict = {}
    for ta in ta_divs:
        nawp_dict['comment_title'] = ta.cssselect('div.title')[0].text_content() #hotel-info 下有很多個span 第一個是area 所以是[0]
        nawp_dict['comment_content'] = ta.cssselect('span')[0].text_content()
        nawp_data.append(nawp_dict)
        db.comment.insert_one({'comment_title':nawp_data[0]['comment_title'] ,'comment_content': nawp_data[0]['comment_content']})

def scrapImage():
    content = browser.page_source # 將 網頁 的原始碼拿出來
    doc = lh.fromstring(content) # content現在還是一個物件 lh = lxml的html ,將它轉成string
    img = browser.find_element_by_css_selector('img.main-photo-image')
    db.images.insert_one({'image_url':img.get_attribute('src')})
   
            
extract()
extract_a()
extract_b()
extract_c()
scrapImage()

next_button = browser.find_element_by_css_selector("a.main-photo-next.next")
test = next_button.get_attribute('style')
# print('test: ', test)
if (test == "display: none;"):
    last_page = True
else:
    last_page = False
    
page_number = 1
while (last_page != True):
    next_button.click()
    time.sleep(5)
    scrapImage()
    page_number += 1 
    print('The page number- %d Extraction is done.'%(page_number))
    test = next_button.get_attribute('style')
    if (test == "display: none;"):
        last_page = True
        print('Data Extraction Done.')
        browser.quit()
    else:
        last_page = False
        next_button = browser.find_element_by_css_selector("a.main-photo-next.next")

print('finish')

The page number- 2 Extraction is done.
The page number- 3 Extraction is done.
The page number- 4 Extraction is done.
The page number- 5 Extraction is done.
The page number- 6 Extraction is done.
The page number- 7 Extraction is done.
The page number- 8 Extraction is done.
The page number- 9 Extraction is done.
The page number- 10 Extraction is done.
The page number- 11 Extraction is done.
The page number- 12 Extraction is done.
The page number- 13 Extraction is done.
The page number- 14 Extraction is done.
The page number- 15 Extraction is done.
The page number- 16 Extraction is done.
The page number- 17 Extraction is done.
The page number- 18 Extraction is done.
The page number- 19 Extraction is done.
The page number- 20 Extraction is done.
The page number- 21 Extraction is done.
The page number- 22 Extraction is done.


ElementNotVisibleException: Message: Element is not currently visible and so may not be interacted with
Stacktrace:
    at fxdriver.preconditions.visible (file:///var/folders/1_/2_2cl49d6_jdxrkrphljstl80000gn/T/tmp4n5qipta/extensions/fxdriver@googlecode.com/components/command-processor.js:10092)
    at DelayedCommand.prototype.checkPreconditions_ (file:///var/folders/1_/2_2cl49d6_jdxrkrphljstl80000gn/T/tmp4n5qipta/extensions/fxdriver@googlecode.com/components/command-processor.js:12644)
    at DelayedCommand.prototype.executeInternal_/h (file:///var/folders/1_/2_2cl49d6_jdxrkrphljstl80000gn/T/tmp4n5qipta/extensions/fxdriver@googlecode.com/components/command-processor.js:12661)
    at DelayedCommand.prototype.executeInternal_ (file:///var/folders/1_/2_2cl49d6_jdxrkrphljstl80000gn/T/tmp4n5qipta/extensions/fxdriver@googlecode.com/components/command-processor.js:12666)
    at DelayedCommand.prototype.execute/< (file:///var/folders/1_/2_2cl49d6_jdxrkrphljstl80000gn/T/tmp4n5qipta/extensions/